## 1 - Top5 de productos más vendidos y vendedor top

### Enunciado

Se requiere identificar cuáles fueron los 5 productos con mayor número de ventas. Además se tiene que determinar cuál fue el vendedor que más unidades vendió de cada uno de los productos del top5.

Para analizar

- ¿Algún vendedor aparece más de una vez como el que más vendió un producto distinto?
- ¿Alguno de estos vendedores representa más del 10% de las ventas totales de ese producto?

---

### Detalles

Se utilizan las tablas:
- `sales`: con los registros de ventas, que incluyen `ProductID`, `SalesPersonID` y `Quantity`.
- `products`: se usa para obtener el nombre del producto.
- `employees`: se usa para obtener el nombre (completo) del vendedor.

Para dar respuesta al enunciado se tiene que:
- Calcular la cantidad total vendida por producto.
- Determinar los 5 productos más vendidos.
- Para cada uno de esos productos, identificar el vendedor con mayor cantidad vendida.
- Calcular qué porcentaje representa esa cantidad sobre el total vendido del producto.

---

### Consultas y resultados

Query

```sql
SELECT
    s.ProductID AS product_id,
    p.ProductName AS product_name,
    SUM(s.Quantity) AS total_quantity
FROM sales s
JOIN products p ON s.ProductID = p.ProductID
GROUP BY s.ProductID, p.ProductName
ORDER BY total_quantity DESC
LIMIT 5;
```

Resultado

![Resultados de la consulta](../assets/ent1-ej1-1.png)

Se tienen identificados los cinco productos más vendidos. 
Los cinco tienen un nivel de ventas similar, cercano a 200k.

---

Query

```sql
WITH 
top5_selling_products AS (
    SELECT
        pr.ProductID,
        pr.ProductName,
        SUM(s.Quantity) AS total_quantity
    FROM sales s
    JOIN products pr ON pr.ProductID = s.ProductID
    GROUP by pr.ProductID, pr.ProductName
    ORDER by total_quantity DESC
    LIMIT 5
),
sales_with_seller_name AS (
    SELECT 
        s.ProductID,
        s.SalesPersonID,
        s.Quantity,
        CONCAT(e.FirstName, ' ', e.LastName) AS seller_name
    FROM sales s
    JOIN employees e ON s.SalesPersonID = e.EmployeeID
),
sales_by_seller_and_product AS (
    SELECT
        s.ProductID, 
        s.SalesPersonID, 
        SUM(s.Quantity) AS quantity_sold,
        s.seller_name
    FROM sales_with_seller_name s
    GROUP BY s.ProductID, s.SalesPersonID, s.seller_name
),
top_seller_per_product AS (
    SELECT 
        ssp.ProductID, 
        ssp.SalesPersonID, 
        ssp.quantity_sold, 
        ssp.seller_name
    FROM 
        sales_by_seller_and_product ssp
    JOIN (
        SELECT 
            ProductID, 
            MAX(quantity_sold) AS max_sold
        FROM sales_by_seller_and_product 
        GROUP BY ProductID
    	) t1 ON ssp.ProductID = t1.ProductID AND ssp.quantity_sold = t1.max_sold
)
SELECT 
    t5sp.ProductID AS product_id,
    t5sp.ProductName AS product_name, 
    tspv.SalesPersonID AS sales_person_id, 
    tspv.seller_name, 
    tspv.quantity_sold AS quantity_sold_by_seller,
    t5sp.total_quantity AS total_quantity_sold  
FROM top5_selling_products t5sp
JOIN top_seller_per_product tspv ON tspv.ProductID = t5sp.ProductID
ORDER BY t5sp.total_quantity DESC;
```

Resultado

![Resultados de la consulta](../assets/ent1-ej1-2.png)

Comentario

- Se observa que los mejores vendedores, de los cinco productos más vendidos, tienen un desempeño similar. 11k ventas por vendedor, de manera aproximada.

---

Query

```sql
WITH 
top5_selling_products AS (
    SELECT
        pr.ProductID,
        pr.ProductName,
        SUM(s.Quantity) AS total_quantity
    FROM sales s
    JOIN products pr ON pr.ProductID = s.ProductID
    GROUP by pr.ProductID, pr.ProductName
    ORDER by total_quantity DESC
    LIMIT 5
),
sales_with_seller_name AS (
    SELECT 
        s.ProductID,
        s.SalesPersonID,
        s.Quantity,
        CONCAT(e.FirstName, ' ', e.LastName) AS seller_name
    FROM sales s
    JOIN employees e ON s.SalesPersonID = e.EmployeeID
),
sales_by_seller_and_product AS (
    SELECT
        s.ProductID, 
        s.SalesPersonID, 
        SUM(s.Quantity) AS quantity_sold,
        s.seller_name
    FROM sales_with_seller_name s
    GROUP BY s.ProductID, s.SalesPersonID, s.seller_name
),
top_seller_per_product AS (
    SELECT 
        ssp.ProductID, 
        ssp.SalesPersonID, 
        ssp.quantity_sold, 
        ssp.seller_name
    FROM 
        sales_by_seller_and_product ssp
    JOIN (
        SELECT 
            ProductID, 
            MAX(quantity_sold) AS max_sold
        FROM sales_by_seller_and_product 
        GROUP BY ProductID
    	) t1 ON ssp.ProductID = t1.ProductID AND ssp.quantity_sold = t1.max_sold
)
SELECT 
    t5sp.ProductID AS product_id,
    t5sp.ProductName AS product_name, 
    tspv.SalesPersonID AS sales_person_id, 
    tspv.seller_name, 
    tspv.quantity_sold AS quantity_sold_by_seller,
    t5sp.total_quantity AS total_quantity_sold,  
    exceeds_ratio_threshold(quantity_sold, total_quantity, 0.1) AS exceeds_10_percent
FROM top5_selling_products t5sp
JOIN top_seller_per_product tspv ON tspv.ProductID = t5sp.ProductID
ORDER BY t5sp.total_quantity DESC;
```

Se definió una función, `exceeds_ratio_threshold`, para determinar si, dado un producot, sus ventas superan o no el 10% de las ventas totales del mismo.

```sql
CREATE FUNCTION exceeds_ratio_threshold(
    sold_quantity INT,
    total_quantity INT,
    threshold DECIMAL(5,4)
)
RETURNS VARCHAR(2)
DETERMINISTIC
BEGIN
    DECLARE result VARCHAR(2);

    IF total_quantity = 0 THEN
        SET result = 'NO';
    ELSEIF (sold_quantity / total_quantity) > threshold THEN
        SET result = 'YES';
    ELSE
        SET result = 'NO';
    END IF;

    RETURN result;
END;
```


Resultado

![Resultados de la consulta](../assets/ent1-ej1-3.png)

Comentarios

- Devon Brewer figura dos veces como el mejor vendedor. 
- Ninguno de los vendedores representa más del 10% de ventas sobre el total vendido de cada producto

---

## 2 - Adopción de los productos más vendidos entre los clientes

### Enunciado

Entre los 5 productos más vendidos, ¿cuántos clientes únicos compraron cada uno y qué proporción representa sobre el total de clientes?  
Analizar si estos productos fueron ampliamente adoptados o si, por el contrario, fueron adquiridos por un grupo reducido que concentró el volumen de ventas.  
Determinar si alguno de los productos depende más de un segmento específico de clientes o no.

--- 

### Consultas y resultados

Query

```sql
WITH top5_products AS (
    SELECT ProductID, SUM(Quantity) AS total_quantity
    FROM sales
    GROUP BY ProductID
    ORDER BY total_quantity DESC
    LIMIT 5
),
unique_customers_per_product AS (
    SELECT t5p.ProductID, COUNT(DISTINCT s.CustomerID) AS unique_customers
    FROM top5_products t5p
    JOIN sales s ON t5p.ProductID = s.ProductID
    GROUP BY t5p.ProductID
),
total_customers AS (
    SELECT COUNT(DISTINCT CustomerID) AS total
    FROM sales
)
SELECT 
    t5p.ProductID as product_id,
    pr.ProductName as product_name,
    t5p.total_quantity as total_quantity_sold,
    ucpp.unique_customers,
    tc.total AS total_customers,
    ROUND(ucpp.unique_customers / tc.total * 100, 2) AS customer_percentage
FROM top5_products t5p
JOIN unique_customers_per_product ucpp ON t5p.ProductID = ucpp.ProductID
JOIN total_customers tc
JOIN products pr ON t5p.ProductID = pr.ProductID
ORDER BY ucpp.unique_customers DESC;
```

Resultado

![Resultados de la consulta](../assets/ent1-ej2-1.png)

Comentarios

- En principo, el volumen de ventas de los productos del top5 se explicaría más por compras frecuentes y/o en cantidad de compra de un grupo reducido de clientes.
- No parece que haya una adopción extendida estos productos. Cerca del 85% de los clientes registrados, no compraron ninguno de los productos del top5.
- No hay diferencias significativas en la proporción de clientes, alrededor de un 15%, que han comprado estos productos. No habría indicios para explicar el nivel de adopción por segmento de clientes.

---

Query

```sql
WITH top_5_products AS (
    SELECT ProductID, SUM(Quantity) AS total_quantity
    FROM sales
    GROUP BY ProductID
    ORDER BY total_quantity DESC
    LIMIT 5
),
sales_with_city AS (
    SELECT 
        s.ProductID,
        s.CustomerID,
        c.CityID
    FROM sales s
    JOIN customers c ON s.CustomerID = c.CustomerID
),
customers_per_product_and_city AS (
    SELECT 
        swc.ProductID,
        swc.CityID,
        COUNT(DISTINCT swc.CustomerID) AS unique_customers
    FROM sales_with_city swc
    JOIN top_5_products t5p ON swc.ProductID = t5p.ProductID
    GROUP BY swc.ProductID, swc.CityID
),
city_distribution_summary AS (
    SELECT 
        cppc.ProductID,
        COUNT(*) AS city_count,  
        MAX(cppc.unique_customers) AS max_customers_in_one_city,
        MIN(cppc.unique_customers) AS min_customers_in_one_city,
        SUM(cppc.unique_customers) AS total_customers_for_product
    FROM customers_per_product_and_city cppc
    GROUP BY cppc.ProductID
)
SELECT 
    p.ProductID,
    pr.ProductName,
    p.total_quantity,
    cds.city_count,
    cds.total_customers_for_product,
    cds.max_customers_in_one_city,
    cds.min_customers_in_one_city,
    ROUND(cds.max_customers_in_one_city * 100.0 / cds.total_customers_for_product, 2) AS max_city_percentage,
    ROUND(cds.min_customers_in_one_city * 100.0 / cds.total_customers_for_product, 2) AS min_city_percentage
FROM top_5_products p
JOIN products pr ON p.ProductID = pr.ProductID
JOIN city_distribution_summary cds ON p.ProductID = cds.ProductID
ORDER BY p.total_quantity DESC;
```

Resultado

![Resultados de la consulta](../assets/ent1-ej2-2.png)

Cometarios

- Se ve que, de los clientes que compraron los productos del top5, están muy repartidos en distintas ciudades. 
- No hay una asociacíon entre el nivel de adopción y la geografía de los compradores.
- Ver que, la ciudad que más/menos compradores tiene para cada producto, ronda el 1.2% / 0.8% de los compradores. 

---


## 3 - Relevancia de los productos más vendidos dentro de sus categorías

### Enunciado

¿A qué categorías pertenecen los 5 productos más vendidos y qué proporción representan dentro del total de unidades vendidas de su categoría?  

---

### Consultas y resultados

Query

```sql
WITH product_sales AS (
    SELECT 
        p.ProductID,
        p.ProductName,
        p.CategoryID,
        c.CategoryName,
        SUM(s.Quantity) AS total_sold
    FROM sales s
    JOIN products p ON p.ProductID = s.ProductID
    JOIN categories c ON c.CategoryID = p.CategoryID
    GROUP BY p.ProductID, p.ProductName, p.CategoryID, c.CategoryName
),
percentage_sales AS (
    SELECT 
        ProductID,
        ProductName,
        CategoryID,
        CategoryName,
        total_sold,
        SUM(total_sold) OVER (PARTITION BY CategoryID) AS category_total,
        RANK() OVER (ORDER BY total_sold DESC) AS global_rank
    FROM product_sales
)
SELECT 
    ProductID,
    ProductName,
    CategoryName,
    total_sold,
    category_total,
    ROUND(total_sold * 100 / category_total, 2) AS category_percentage,
    global_rank
FROM percentage_sales
ORDER BY global_rank
LIMIT 5;
```
Resultado

![Resultados de la consulta](../assets/ent1-ej3-1.png)

Cometarios

- Los cinco productos más vendidos pertenecen a categías distitnas.
- Representan entre 2% y 3% de las ventas de la propia categoría.
- Ccomo no se repiten, ni categoría ni producto, es claro que, cada uno de ellos es el mas vendido dentro de su categoría.

---

## 4 - Ranking global y por categoría de los productos más vendidos

### Enunciado

¿Cuáles son los 10 productos con mayor cantidad de unidades vendidas en todo el catálogo y cuál es su posición dentro de su propia categoría?  
  
El objetivo es analizar si estos productos también son líderes en su categoría o si existen competidores cercanos, y observar cómo se concentra la demanda en determinadas categorías.

---

### Consultas y resultados

Query

```sql
WITH product_sales AS (
    SELECT 
        p.ProductID,
        p.ProductName,
        p.CategoryID,
        c.CategoryName,
        SUM(s.Quantity) AS total_sold
    FROM sales s
    JOIN products p ON p.ProductID = s.ProductID
    JOIN categories c ON c.CategoryID = p.CategoryID
    GROUP BY p.ProductID, p.ProductName, p.CategoryID, c.CategoryName
),
sales_with_rankings AS (
    SELECT 
        *,
        RANK() OVER (ORDER BY total_sold DESC) AS global_rank,
        RANK() OVER (PARTITION BY CategoryID ORDER BY total_sold DESC) AS category_rank
    FROM product_sales
)
SELECT 
    ProductID,
    ProductName,
    CategoryName,
    total_sold,
    global_rank,
    category_rank
FROM sales_with_rankings
WHERE global_rank <= 15
ORDER BY global_rank;
```

Resultado

![Resultados de la consulta](../assets/ent1-ej4-1.png)

Cometarios

- Para tres de las categorías, Snails, Meat y Poultry, figuran en el top10 de productos más vendidos, el primer y segundo producto de dichas categorías. 
- Sin embargo, la diferencia del volumen de ventas entre primer y segundo producto de estas categorías, es muy chica. Al menos, enntre los más vendidos, hay una fuerte competencia 
- Si se mira un top15, se ve que otra categorías empiezan a tener más de un producto entre los más vendidos y también, que de las tres que repiten en el top10, empiezan a tener más de dos productos repetidos pero no se suma ninguna nueva categoría.
- Se tiene entonces, que los productos más vendidos provienen de 7 categorías que, de manera bastante equilibrada y con reformance de ventas similares, dominan en ventas.
---